# Experiment F

2) Implementar um framework de busca de hiperparâmetros.

2.1) Parâmetros específicos para cada método de processamento do ds.

2.2) N top colunas (`top_cols`) do dataset.

2.3) Parâmetro $L$ (`recomender(...,L,...)`).

5) Implementar como entrada uma empresa nova, conter mapeamento de valores.

Talvez criar o notebook da Second_View, com:
1) Verificar `sklearn.inspection.permutation_importance`.

- Author: Israel Oliveira [\[e-mail\]](mailto:'Israel%20Oliveira%20'<prof.israel@gmail.com>)

In [1]:
%load_ext watermark

In [2]:
from typing import NewType, List
from loguru import logger
import pandas as pd
import numpy as np
from time import time
from scipy.spatial.distance import cosine
from sklearn.decomposition import FactorAnalysis, FastICA, PCA, IncrementalPCA, NMF, TruncatedSVD
from collections import defaultdict, Counter
import functools
import operator
from copy import deepcopy
from tqdm import tqdm


In [3]:
# Run this cell before close.
%watermark -d --iversion -b -r -g -m -v

numpy  1.19.0
pandas 1.0.5
2020-07-30 

CPython 3.7.8
IPython 7.16.1

compiler   : GCC 8.3.0
system     : Linux
release    : 5.4.0-7634-generic
machine    : x86_64
processor  : 
CPU cores  : 8
interpreter: 64bit
Git hash   : 87718a215d70826711cb0d8123e4673cc1e76f45
Git repo   : https://github.com/ysraell/aceleradev_private.git
Git branch : master


In [4]:
def Manhattan(X,vec):
    return abs(X - vec).sum(-1)

def Camberra(X,vec):
    return abs((X - vec)/(X + vec)).sum(-1)

def BrayCurtis(X,vec):
    return abs((X - vec)).sum(-1) / abs((X - vec)).sum(-1).sum(-1)

def np_cossine(X,vec):
    return np.array([sum(X[i]*vec) / sum(X[i]**2)*sum(vec**2) for i in range(X.shape[0])])

def npj_cossine(X,vec):
    return npj.array([sum(X[i]*vec) / sum(X[i]**2)*sum(vec**2) for i in range(X.shape[0])])

def scy_cossine(X,vec):
    return np.array([cosine(X[i],vec) for i in range(X.shape[0])])

dist_func = [Manhattan, Camberra, BrayCurtis, np_cossine, scy_cossine]

def Nothing(arg):
    return arg

def npSVD(M):
    u, _, _ = np.linalg.svd(M, full_matrices=False)
    return u

def _PCA(M,n_components=None):
    out = PCA(n_components=n_components)
    return out.fit_transform(M)

def _FastICA(M,n_components=None):
    out = FastICA(n_components=n_components)
    return out.fit_transform(M)

def _FactorAnalysis(M,n_components=None):
    out = FactorAnalysis(n_components=n_components)
    return out.fit_transform(M)

def _IncrementalPCA(M,n_components=None):
    out = IncrementalPCA(n_components=n_components)
    return out.fit_transform(M)

def _TruncatedSVD(M,n_components=None):
    out = TruncatedSVD(n_components=n_components)
    return out.fit_transform(M)

def _NMF(M,n_components=None):
    out = NMF(n_components=n_components)
    return out.fit_transform(M)

redux_func = [Nothing, npSVD, _NMF, _TruncatedSVD, _IncrementalPCA, _FactorAnalysis, _FastICA, _PCA]

def escalaropt_missings(df: pd.DataFrame, score: dict):
    df_score = pd.DataFrame(score.items(), columns=['col','score'])
    df_score['escala_opt'] = 1-normalize((np.sqrt(df_score.score)))
    df_score['escala_opt'] = df_score['escala_opt'].apply(lambda x: max(x,0.1))
    for _,row in df_score.iterrows():
        df[row.col] = row.escala_opt*df[row.col]
    return df

def escalaropt_std(df: pd.DataFrame, score: dict):
    df_score = pd.DataFrame(score.items(), columns=['col','score'])
    df_score['escala_opt'] = normalize([np.sqrt(np.sqrt(np.sqrt(df[col].std()))) for col in df_score['col']])
    df_score['escala_opt'] = df_score['escala_opt'].apply(lambda x: max(x,0.1))
    for _,row in df_score.iterrows():
        df[row.col] = row.escala_opt*df[row.col]
    return df

def escalaropt_entropy(df: pd.DataFrame, score: dict):
    df_score = pd.DataFrame(score.items(), columns=['col','score'])
    df_score['escala_opt'] = normalize([(-sum((df[col]+1)*np.log(df[col]+1))) for col in df_score['col']])
    df_score['escala_opt'] = df_score['escala_opt'].apply(lambda x: max(x,0.1))
    for _,row in df_score.iterrows():
        df[row.col] = row.escala_opt*df[row.col]
    return df

def scalarop_nothing(arg, arg2):
    return arg

procDS_func = [scalarop_nothing, escalaropt_missings, escalaropt_std, escalaropt_entropy]



In [5]:
!ls ../app

Usage.ipynb  data  src


In [6]:
import sys
sys.path.insert(1, '../app/src/')
from train import *
from recommender import Recommender 

from tqdm import tqdm
import pandas as pd
from time import time

In [7]:
df_train, df_test = load_dataset(path_data = '../app/data/', test_list = [0, 1], train_test_merged = False)
ds, score = feat_proc(df_train)

2020-07-30 02:01:31.435 | INFO     | utils:load_dataset:82 - Carregando dataset de treino...
2020-07-30 02:01:45.507 | INFO     | utils:load_dataset:91 - ...pronto!
2020-07-30 02:01:45.508 | INFO     | utils:load_dataset:95 - Carregando dataset de teste...
2020-07-30 02:01:45.521 | INFO     | utils:load_dataset:104 - ...pronto!
2020-07-30 02:01:45.522 | INFO     | utils:feat_proc:128 - Processando as features...
2020-07-30 02:02:22.270 | INFO     | utils:feat_proc:163 - ...pronto!


In [8]:
def Search(N=1, process_values = Nothing, factorize = Nothing, vector_distance_list = [Manhattan]):
    ex_algo = ExMatrix(process_values = process_values, factorize = factorize)
    ex_algo.fit(ds, score)
    out = {}
    for dist in vector_distance_list:
        ex_algo.vector_distance = dist
        print(dist.__name__)
        tmp ={1: [], 2: []}
        t = time()
        for row in tqdm(df_test.iterrows()):
            recs = ex_algo.recomender([row[1].id],k=N)
            tmp[row[1].P].append(any([x in df_test.loc[df_test.P == row[1].P].id.to_list() for x in recs])*1)
        t = time()-t
        out[dist.__name__] = {i: (sum(val)/max(1,len(val)), sum(val), len(val)) for i,val in tmp.items()}
        out[dist.__name__]['t'] = t
    return out

In [9]:
dist_list = [Manhattan] #[Manhattan, scy_cossine, Camberra, BrayCurtis, np_cossine] #[Manhattan, Camberra, BrayCurtis] #dist_func
proc_list = [scalarop_nothing, escalaropt_missings] #procDS_func #[Nothing] #procDS_func
redux_list = redux_func #redux_func #[Nothing] #[Nothing, npSVD, _NMF, _PCA, _FactorAnalysis] #redux_func
n_components_dict = {Nothing.__name__ : False,
                  #_npSVDj.__name__: False,
                  npSVD.__name__: False,
                  _NMF.__name__ : True,
                  _TruncatedSVD.__name__ : True,
                  _IncrementalPCA.__name__ : True,
                  _FactorAnalysis.__name__ : True,
                  _FastICA.__name__ : True,
                  _PCA.__name__ : True}

In [10]:
results_csv_name = 'Results_A1.csv'

In [11]:
df_laoded = pd.read_csv(results_csv_name)

In [12]:
def save_results(results,df_e):
    df = pd.DataFrame(results, columns=['pre_proc','redux_func','n_components','dist','t','P1pp','P1_True','P1_len','P2pp','P2_True','P2_len'])
    out = pd.concat([df_e, df])
    #df_laoded.df_laodedto_csv('Results_redux_prepro.csv',index=False)
    return out

In [13]:
n_components_list = [n for n in range(10,160,10)]
n=0
for redux in redux_list:
    for proc in proc_list:
        if n_components_dict[redux.__name__]:
            for n_components in n_components_list:
                cond = (df_laoded['pre_proc'] == proc.__name__) & (df_laoded['redux_func'] == redux.__name__) & (df_laoded['n_components'] == n_components)
                print("Done?: {}.".format([proc.__name__, redux.__name__, n_components]))
                if (sum(cond) == 0) or (not all([d.__name__ in df_laoded.dist.loc[cond].unique() for d in dist_list])):
                    dist_list_tmp = [d for d in dist_list if d.__name__ not in df_laoded.dist.loc[cond].unique()]
                    def redux_tmp(M):
                        return redux(M,n_components=n_components)
                    tmp = Search(process_values = proc, factorize = redux_tmp, vector_distance_list = dist_list_tmp)
                    results = [[proc.__name__, redux.__name__, n_components] + r for r in [[key]+[tmp[key]['t']]+flat([list(tmp[key][i+1]) for i in range(2)]) for key in tmp.keys()]]
                    df_laoded = save_results(results,df_laoded)
                    df_laoded.to_csv(results_csv_name,index=False)
                else:
                    print("Done: {}.".format([proc.__name__, redux.__name__, n_components]))
        else:
            n_components = ds.shape[1]
            cond = (df_laoded['pre_proc'] == proc.__name__) & (df_laoded['redux_func'] == redux.__name__) & (df_laoded['n_components'] == n_components)
            print("Done?: {}.".format([proc.__name__, redux.__name__, n_components]))
            if (sum(cond) == 0) or (not all([d.__name__ in df_laoded.dist.loc[cond].unique() for d in dist_list])):
                dist_list_tmp = [d for d in dist_list if d.__name__ not in df_laoded.dist.loc[cond].unique()]
                tmp = Search(process_values = proc, factorize = redux, vector_distance_list= dist_list_tmp)
                results = [[proc.__name__, redux.__name__, n_components] + r for r in [[key]+[tmp[key]['t']]+flat([list(tmp[key][i+1]) for i in range(2)]) for key in tmp.keys()]]
                df_laoded = save_results(results,df_laoded)
                df_laoded.to_csv(results_csv_name,index=False)
            else:
                print("Done: {}.".format([proc.__name__, redux.__name__, n_components]))

Done?: ['scalarop_nothing', 'Nothing', 169].
Done: ['scalarop_nothing', 'Nothing', 169].
Done?: ['escalaropt_missings', 'Nothing', 169].
Done: ['escalaropt_missings', 'Nothing', 169].
Done?: ['scalarop_nothing', 'npSVD', 169].
Done: ['scalarop_nothing', 'npSVD', 169].
Done?: ['escalaropt_missings', 'npSVD', 169].
Done: ['escalaropt_missings', 'npSVD', 169].
Done?: ['scalarop_nothing', '_NMF', 10].
Done: ['scalarop_nothing', '_NMF', 10].
Done?: ['scalarop_nothing', '_NMF', 20].
Done: ['scalarop_nothing', '_NMF', 20].
Done?: ['scalarop_nothing', '_NMF', 30].
Done: ['scalarop_nothing', '_NMF', 30].
Done?: ['scalarop_nothing', '_NMF', 40].
Done: ['scalarop_nothing', '_NMF', 40].
Done?: ['scalarop_nothing', '_NMF', 50].
Done: ['scalarop_nothing', '_NMF', 50].
Done?: ['scalarop_nothing', '_NMF', 60].
Done: ['scalarop_nothing', '_NMF', 60].
Done?: ['scalarop_nothing', '_NMF', 70].
Done: ['scalarop_nothing', '_NMF', 70].
Done?: ['scalarop_nothing', '_NMF', 80].
Done: ['scalarop_nothing', '_NMF

2020-07-30 02:02:23.284 | INFO     | model:fit:69 - Processando valores.
2020-07-30 02:02:23.422 | INFO     | model:fit:73 - Fatorizando.
2020-07-30 02:03:25.657 | INFO     | model:fit:78 - Matriz pronta.
0it [00:00, ?it/s]

Manhattan


831it [11:41,  1.18it/s]
2020-07-30 02:15:07.460 | INFO     | model:fit:69 - Processando valores.


Done?: ['scalarop_nothing', '_FastICA', 50].


...

831it [13:25,  1.03it/s]
2020-07-30 16:09:42.358 | INFO     | model:fit:69 - Processando valores.


Done?: ['escalaropt_missings', '_PCA', 150].


2020-07-30 16:09:42.732 | INFO     | model:fit:73 - Fatorizando.
2020-07-30 16:09:52.459 | INFO     | model:fit:78 - Matriz pronta.
0it [00:00, ?it/s]

Manhattan


831it [13:13,  1.05it/s]


In [ ]:
#df = pd.DataFrame(columns=['pre_proc','redux_func','n_components','dist','t','P1pp','P1_True','P1_len','P2pp','P2_True','P2_len'])
#df.to_csv(results_csv_name,index=False)